# A/B Test Analysis
We're going to conduct an Independent Samples T-test to analyse our A/B test. An Indepdent Samples T-test compares the differences between two means of two different samples

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

Export your results to a .csv file and save it to you github repository. Import your .csv file, inspect it, and clean it where neccesary.

In [2]:
# If on Github, load your data
df_A = pd.read_csv("/Users/celinewu/Documents/GitHub/2023-24c-fai1-adsai-CelineWu231265/AB_test/Version_A - AB-test_HCAI_Version_A(Sheet1).csv")
df_B = pd.read_csv("/Users/celinewu/Documents/GitHub/2023-24c-fai1-adsai-CelineWu231265/AB_test/Version_B - AB-test_HCAI_Version_B(Sheet1).csv")

# EDA A
df_A.info() # Is your data in the right format?
df_A.head() # Quick EDA. No? Clean it, you only want the rows and columns containing likert-score data, saved as integers.

# EDA B
df_B.info() # Is your data in the right format?
df_B.head() # Quick EDA. No? Clean it, you only want the rows and columns containing likert-score data, saved as integers.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Id                                          16 non-null     int64
 1   I understood what I could use the app for.  16 non-null     int64
 2   I found the application intuitive to use.   16 non-null     int64
 3   I thought the application was useful.       16 non-null     int64
 4   I enjoyed using the application.            16 non-null     int64
 5   The app flow was self explanatory           16 non-null     int64
dtypes: int64(6)
memory usage: 896.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Id                                    

,Id,I understood what I could use the app for.,I found the application intuitive to use.,I thought the application was useful.,I enjoyed using the application.,The app flow was self explanatory
0,1,1,1,1,1,1
1,2,6,1,1,1,1
2,3,5,5,5,5,5
3,4,6,5,5,5,6
4,5,7,7,7,7,7


The rest we leave for tomorrow when we actually have our data. But if you are eager to play around a bit you can simply refresh the survey and fill in a couple of responses to create an A and a B version.

Now, let's start analysing our gathered data! This block we won't dive into inferential statistics since it can get complex quite fast; we'll do that in Year 2, block A. For now, you just need to know that we need to test whether the data is normally distributed and whether the variances of both samples are equal. Otherwise, our statistical tests would not be valid and we can therefore not say that the results we're seeing are due to chance. What we are going to statistically ascertain is whether there is a statistically significant different in the mean of a given variable for version A or B. 

# Question 1

In [4]:
# Run the shaprio-wilk statistical test for each question to check whether the data is normally distributed
normal_a = stats.shapiro(df_A["I understood what I could use the app for."])
normal_b = stats.shapiro(df_B["I understood what I could use the app for."])

# Check whether the variance of both samples is equal
homogeneity = stats.levene(df_A["I understood what I could use the app for."],
                           df_B["I understood what I could use the app for."])

print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_a, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_b, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the groups have equal variances", homogeneity, ". If the variance aren't equal then you will have to run the bootstrapped version.")

If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.7951330258826621, pvalue=0.0023486318428322956) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.6664171981299241, pvalue=5.290371493857222e-06) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the groups have equal variances LeveneResult(statistic=0.3357001972386588, pvalue=0.5658330056280033) . If the variance aren't equal then you will have to run the bootstrapped version.


Now that is in the right format and we know the column names. Replace 'A' with the column name which holds your original baseline version; A. Replace 'B' with the column name which holds the result of your improved version; B.

In [5]:
# Run Independent Samples T-test when assumptions are not violated.
results = stats.ttest_ind(df_A["I understood what I could use the app for."],
                          df_B["I understood what I could use the app for."])

# Print the results
print(f"The results are significant if the p-value is significant, which means smaller than 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means smaller than 0.05 TtestResult(statistic=-0.4951507864823368, pvalue=0.6234215810963804, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


In [6]:
# Run Bootstrapped Independent Samples T-test when assumptions are violated
rng = np.random.default_rng() # create random sampling

results = stats.ttest_ind(df_A["I understood what I could use the app for."],
                          df_B["I understood what I could use the app for."],
                          random_state = rng)

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means => 0.05 TtestResult(statistic=-0.4951507864823368, pvalue=0.6234215810963804, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


Great, that was our first t-test. Save the results to your learning log in the week 8 and interpret them there. Were they what you expected? What are you going to change to improve your design if neccesary. 

# Question 2

In [7]:
# Run the shaprio-wilk statistical test for each question to check whether the data is normally distributed
normal_a = stats.shapiro(df_A["I found the application intuitive to use."])
normal_b = stats.shapiro(df_B["I found the application intuitive to use."])

# Check whether the variance of both samples is equal
homogeneity = stats.levene(df_A["I found the application intuitive to use."],
                           df_B["I found the application intuitive to use."])

print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_a, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_b, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the groups have equal variances", homogeneity, ". If the variance aren't equal then you will have to run the bootstrapped version.")

If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.7773626318229307, pvalue=0.001384357498053207) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.7159623485779187, pvalue=2.2712078213567356e-05) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the groups have equal variances LeveneResult(statistic=0.8079341284946717, pvalue=0.37454430351244306) . If the variance aren't equal then you will have to run the bootstrapped version.


In [8]:
# Run Independent Samples T-test when assumptions are not violated.
results = stats.ttest_ind(df_A["I found the application intuitive to use."],
                          df_B["I found the application intuitive to use."])

# Print the results
print(f"The results are significant if the p-value is significant, which means smaller than 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means smaller than 0.05 TtestResult(statistic=-1.0472272162506624, pvalue=0.3017879146791575, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


In [11]:
# Run Bootstrapped Independent Samples T-test when assumptions are violated
rng = np.random.default_rng() # create random sampling

results = stats.ttest_ind(df_A["I found the application intuitive to use."],
                          df_B["I found the application intuitive to use."],
                          random_state = rng)

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means => 0.05 TtestResult(statistic=-1.0472272162506624, pvalue=0.3017879146791575, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


# Question 3

In [13]:
# Run the shaprio-wilk statistical test for each question to check whether the data is normally distributed
normal_a = stats.shapiro(df_A["I\xa0thought the application was useful."])
normal_b = stats.shapiro(df_B["I thought the application was useful."])

# Check whether the variance of both samples is equal
homogeneity = stats.levene(df_A["I\xa0thought the application was useful."],
                           df_B["I thought the application was useful."])

print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_a, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_b, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the groups have equal variances", homogeneity, ". If the variance aren't equal then you will have to run the bootstrapped version.")

If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.8219270437525785, pvalue=0.005406418999061028) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.722876615752247, pvalue=2.8139761873641475e-05) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the groups have equal variances LeveneResult(statistic=0.022678144886886763, pvalue=0.8811150663763754) . If the variance aren't equal then you will have to run the bootstrapped version.


In [14]:
# Run Independent Samples T-test when assumptions are not violated.
results = stats.ttest_ind(df_A["I\xa0thought the application was useful."],
                          df_B["I thought the application was useful."])

# Print the results
print(f"The results are significant if the p-value is significant, which means smaller than 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means smaller than 0.05 TtestResult(statistic=-0.7821579499548239, pvalue=0.43909882379629805, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


In [15]:
# Run Bootstrapped Independent Samples T-test when assumptions are violated
rng = np.random.default_rng() # create random sampling

results = stats.ttest_ind(df_A["I\xa0thought the application was useful."],
                          df_B["I thought the application was useful."],
                          random_state = rng)

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means => 0.05 TtestResult(statistic=-0.7821579499548239, pvalue=0.43909882379629805, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


# Question 4


In [16]:
# Run the shaprio-wilk statistical test for each question to check whether the data is normally distributed
normal_a = stats.shapiro(df_A["I enjoyed using the application."])
normal_b = stats.shapiro(df_B["I enjoyed using the application."])

# Check whether the variance of both samples is equal
homogeneity = stats.levene(df_A["I enjoyed using the application."],
                           df_B["I enjoyed using the application."])

print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_a, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_b, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the groups have equal variances", homogeneity, ". If the variance aren't equal then you will have to run the bootstrapped version.")

If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.8470651452461548, pvalue=0.012338410326657763) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.722876615752247, pvalue=2.8139761873641475e-05) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the groups have equal variances LeveneResult(statistic=0.0004943814219478673, pvalue=0.9823802266701495) . If the variance aren't equal then you will have to run the bootstrapped version.


In [17]:
# Run Independent Samples T-test when assumptions are not violated.
results = stats.ttest_ind(df_A["I enjoyed using the application."],
                          df_B["I enjoyed using the application."])

# Print the results
print(f"The results are significant if the p-value is significant, which means smaller than 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means smaller than 0.05 TtestResult(statistic=-1.3262617564193029, pvalue=0.19288199186454708, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


In [18]:
# Run Bootstrapped Independent Samples T-test when assumptions are violated
rng = np.random.default_rng() # create random sampling

results = stats.ttest_ind(df_A["I enjoyed using the application."],
                          df_B["I enjoyed using the application."],
                          random_state = rng)

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means => 0.05 TtestResult(statistic=-1.3262617564193029, pvalue=0.19288199186454708, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


# Question 5

In [19]:
# Run the shaprio-wilk statistical test for each question to check whether the data is normally distributed
normal_a = stats.shapiro(df_A["The app flow was self explanatory\xa0"])
normal_b = stats.shapiro(df_B["The app flow was self explanatory "])

# Check whether the variance of both samples is equal
homogeneity = stats.levene(df_A["The app flow was self explanatory\xa0"],
                           df_B["The app flow was self explanatory "])

print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_a, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the data is normally distrubted", normal_b, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the groups have equal variances", homogeneity, ". If the variance aren't equal then you will have to run the bootstrapped version.")

If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.8151405846534079, pvalue=0.004358231208181672) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the data is normally distrubted ShapiroResult(statistic=0.6977170438686422, pvalue=1.3077230328684643e-05) . If the data is not normally distributed then you will have to run the bootstrapped version.
If the p-value is above 0.05, then the groups have equal variances LeveneResult(statistic=0.08473443701747307, pvalue=0.7726067242135081) . If the variance aren't equal then you will have to run the bootstrapped version.


In [20]:
# Run Independent Samples T-test when assumptions are not violated.
results = stats.ttest_ind(df_A["The app flow was self explanatory\xa0"],
                          df_B["The app flow was self explanatory "])

# Print the results
print(f"The results are significant if the p-value is significant, which means smaller than 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means smaller than 0.05 TtestResult(statistic=-1.7576929406677801, pvalue=0.08707134844947921, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.


In [21]:
# Run Bootstrapped Independent Samples T-test when assumptions are violated
rng = np.random.default_rng() # create random sampling

results = stats.ttest_ind(df_A["The app flow was self explanatory\xa0"],
                          df_B["The app flow was self explanatory "],
                          random_state = rng)

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means => 0.05 TtestResult(statistic=-1.7576929406677801, pvalue=0.08707134844947921, df=37.0) 
 If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.
